## What is Matrix Factorization?

- **Matrix factorization** is a mathematical technique commonly used in recommendation systems to factorize a user-item interaction matrix into lower-dimensional matrices representing users and items.
- The core concept behind matrix factorization is to approximate the original matrix by decomposing it into two or more matrices.
- This process helps uncover latent factors that influence user-item interactions.


### Let's Breakdown the process of Matrix Factorization in Simple Steps

- User-Item Interaction Matrix:

 - In recommendation systems, you start with a user-item interaction matrix, often denoted as R.
 - Rows correspond to users, columns correspond to items, and the entries $R_{i,j}$ represent user ratings, interactions, or preferences for items.

 <center><img src="https://drive.google.com/uc?id=1gGxM-opp1666trtmfMlc6JY4Ue46imbn"></center>

- Decomposition:

 - Matrix factorization decomposes this user-item interaction matrix R into two lower-dimensional matrices, typically denoted as U (user matrix) and I (item matrix).

 <center><img src="https://drive.google.com/uc?id=1t3poDcWoe-ewAi_QCgEXe1SBoSgqZGOB"></center>

- Dimensions:

 - The dimensions of the user matrix U are M x K, where M is the number of users, and K is the number of latent factors. The item matrix I has dimensions of K x N, where N is the number of items.

- Objective Function:

 - The goal of matrix factorization is to find the matrices U and V such that the product U * V approximates the original matrix R.
 - To achieve this, an objective function is defined, often using a loss function like Mean Squared Error (MSE) or a variant of it.

 > $Loss = 𝚺 (R_{ij}-(U_{i}*I_{j}))^{2}$

 - Here, U_i and I_j are the latent factor vectors for the i-th user and j-th item, respectively.

- Optimization:

 - The matrices U and I are optimized to minimize the loss function. This is typically done using optimization techniques like Gradient Descent or Alternating Least Squares (ALS).

 - We'll use Gradient descent to optimize the loss function.

 - The optimization process updates the latent factor vectors in U and V iteratively to improve the approximation of the original matrix R.
 - The objective is to find U and V that provide the best fit to the observed user-item interactions.

- Prediction:

 - Once the optimization is complete, the factorized matrices U and V can be used to predict missing values in the original matrix R.
 - These predictions are used to generate recommendations for users.





In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1' -O "movies.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1HOFWUAMFlYbd-gk1B2IyV2-hXDZI7gKR' -O "ratings.csv"
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1b7_yRRBs3s3atp1WQHN2GU577vxY8u_h' -O "users.csv"



--2025-12-10 09:00:09--  https://drive.google.com/uc?export=download&id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1
Resolving drive.google.com (drive.google.com)... 74.125.202.138, 74.125.202.100, 74.125.202.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.202.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download [following]
--2025-12-10 09:00:09--  https://drive.usercontent.google.com/download?id=1Q9UJtrN_v_dS-garl5gQ1I_SotGhye_1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.195.132, 2607:f8b0:4001:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.195.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 515699 (504K) [application/octet-stream]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 503.61K  --.-KB/s   

In [ ]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
users = pd.read_csv("users.csv")

print(movies.shape)
print(ratings.shape)
print(users.shape)



(10329, 3)
(105339, 4)
(668, 3)


In [ ]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [ ]:
users.head(3)

,userId,age,time_spent_per_day
0,1,16,3.976315
1,2,24,1.891303
2,3,20,4.521478


In [ ]:
# ratings.head(3)
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head(3)


,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0


- Converting the original user-item interaction data into a user-item matrix where rows represent users, columns represent movies, and the entries contain user ratings.
- Any missing values (unrated movies) are filled with 0, creating a matrix suitable for collaborative filtering in recommendation systems.

In [ ]:
# index - users
# columns - movies
# values - ratings

In [ ]:
rm = ratings.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
rm.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,144482,144656,144976,146344,146656,146684,146878,148238,148626,149532
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Matrix factorization from Scratch

In [ ]:
rm_small = rm.copy()
rm_small = rm_small[rm_small.columns[:100]] # First 100 movies/columns
rm_small = rm_small.head(100) # First 100 users/rows

rm_small.head()



movieId,1,2,3,4,5,6,7,8,9,10,...,100,101,102,103,104,105,107,108,110,111
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,5.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


- Matrix factorization using stochastic gradient descent to minimize the cost function.
- It iteratively optimizes latent factor matrices P and Q to approximate the user-item interaction matrix R. After optimization, it can predict user-item interactions and compares them with actual ratings.

In [ ]:
K = 2 #Latent factors
P = np.random.normal(size = (rm_small.shape[0], K))
Q = np.random.normal(size = (rm_small.shape[1], K))

def matrix_factorization(R, P, Q, K, steps = 10000, alpha = 0.0002, beta = 0.02):

  Q = Q.T

  for step in range(steps):
    for i in range(len(R)): #loop through all users
      for j in range(len(R[i])): #loop through all movies of that user

        if R[i][j] == 0:
          continue

        eij = R[i][j] - np.dot(P[i,:], Q[:,j]) # Calculate predicted error

        for k in range(K):
          x = P[i][k]

          P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
          Q[k][j] += alpha * (2 * eij * x - beta * Q[k][j])

  return P, Q.T



In [ ]:
B, C = matrix_factorization(rm_small.values, P.copy(), Q.copy(), 2)

In [ ]:
print(B)

[[ 1.44283391e+00 -1.16830568e+00]
 [ 1.68385098e+00  1.23037021e+00]
 [ 2.38607511e+00 -1.70723454e-01]
 [ 2.22038491e+00  9.63368936e-01]
 [ 5.26413393e-01  3.25996200e+00]
 [-1.00639572e+00 -1.70699179e+00]
 [ 2.21602772e+00  2.76481909e-01]
 [ 1.70273876e+00  1.16541147e+00]
 [ 1.56418258e+00  4.80743505e-01]
 [ 3.01808098e+00 -1.74107359e+00]
 [ 1.62787019e+00 -7.07188840e-02]
 [ 1.29394703e+00  1.18255704e+00]
 [ 1.70813635e+00 -3.99082890e-01]
 [ 1.31551773e+00  1.55901058e+00]
 [ 2.03971063e+00 -6.55426424e-01]
 [ 2.37770451e+00  1.45180841e+00]
 [ 1.81424397e+00  2.59842864e-02]
 [ 1.58626218e+00 -1.31001081e+00]
 [ 2.83343261e-01  3.22960930e-01]
 [-3.73453129e-01  3.09275293e-01]
 [ 1.74012065e+00 -1.90356024e+00]
 [ 1.61755065e+00  4.37867441e-01]
 [ 1.44289280e+00 -1.05469764e+00]
 [ 2.29209329e+00  5.90289152e-01]
 [ 1.38154044e+00  8.83680056e-01]
 [ 1.25039200e+00 -1.33080418e+00]
 [ 4.65951831e-01  1.11650781e+00]
 [ 2.20236803e+00 -1.04962313e+00]
 [ 1.67356907e+00  2

In [ ]:
print(C)

[[ 2.21499102  0.8637885 ]
 [ 1.85640098 -0.76160982]
 [ 1.60919107  0.51047692]
 [ 0.99931188  1.17001826]
 [ 1.47517057  0.6654577 ]
 [ 2.20442927 -0.50272914]
 [ 1.29122576  0.89770897]
 [-1.1143845   2.14916909]
 [ 0.75885149  1.51529946]
 [ 1.83973129  1.00700071]
 [ 1.85657337  1.06856822]
 [ 1.76871359 -0.25920267]
 [ 0.90876826 -0.09898976]
 [ 1.26663758  1.41361808]
 [ 1.85809717  1.02155499]
 [ 2.1249136  -0.9069073 ]
 [ 2.04972837  0.8302644 ]
 [ 0.70439497  2.23255585]
 [ 1.55155104 -0.31874999]
 [ 1.86653973 -0.92471027]
 [ 1.94163522 -0.40792094]
 [ 1.94146954 -0.58226184]
 [ 1.67818262  0.66088584]
 [ 2.03099283  0.89348604]
 [ 2.24577957 -0.05647988]
 [ 1.89090897 -0.99464834]
 [-0.24977434  1.39547202]
 [ 1.81095767 -0.92222983]
 [ 2.53881186 -0.82898624]
 [ 1.25269815  1.27037372]
 [ 1.557113    1.18608383]
 [ 2.29033917 -0.56039665]
 [ 0.80435301  0.10881448]
 [ 1.81954822  1.02345052]
 [ 0.61276886 -0.67564158]
 [ 1.72989404  1.65531538]
 [ 1.797584    0.38382583]
 

In [ ]:
# Predicted Values vs Actual Ratings

print(np.dot(B[4], C[36]), rm_small.values[4, 36])
print(np.dot(B[1], C[0]), rm_small.values[1, 0])
print(np.dot(B[1], C[2]), rm_small.values[1, 2])
print(np.dot(B[3], C[17]), rm_small.values[3, 17])



2.197529902164662 0.0
4.792494426091629 5.0
3.3377135589399742 2.0
3.7148029166083933 0.0


In [ ]:
print(rm_small.shape)


(100, 100)


In [ ]:
print(rm_.shape)


(668, 10325)


In [ ]:
from sklearn.metrics import mean_squared_error as mse

rm_ = np.dot(B, C.T)
print(mse(rm_small.values, rm_)) # Predicted ratings matrix

6.979475600405753


In [ ]:
mse(rm_small.values[rm_small > 0], rm_[rm_small > 0])
# To filter only known ratings from original and predicted matrix

0.3412234649079463

# Using cmfrec library -> Collective matrix factorization in recommender systems

- cmfrec library requires input in the form of dataframe not as sparse matrix.

- It requires 3 columns UserId, ItemId, rating.

In [ ]:
ratings.head(2)

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5


In [ ]:
rm_raw = ratings.copy()
rm_raw.head()

,userId,movieId,rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [ ]:
rm_raw.shape

(105339, 3)

In [ ]:
!pip install cmfrec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached findblas-0.1.26.post1-py3-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-3.5.1.post13-cp312-cp312-linux_x86_64.whl size=5975939 sha256=15a8d84c8fdd027d87c3cc859f05f50d69c8cb7c999b7824d22a26584f097e4c
  Stored in directory: /root/.cache/pip/wheels/46/9d/a7/03614a1633b40dde9aeb6eb12265f9c684feba1e3d82fe05bf
Successfully built cmfrec


In [ ]:
from cmfrec import CMF #Collective Matrix Factorization

In [ ]:
rm_raw.rename(columns = {'userId': 'UserId', 'movieId':'ItemId', 'rating':'Rating'}, inplace = True)

In [ ]:
rm_raw.head()

,UserId,ItemId,Rating
0,1,16,4.0
1,1,24,1.5
2,1,32,4.0
3,1,47,4.0
4,1,50,4.0


In [ ]:
df = rm_raw.copy()

model = CMF(
    method="als",
    k=2,
    lambda_=0.1,
    user_bias=False,
    item_bias=False,
    verbose=False
)

**An instance of the CMF model is created with various hyperparameters:**

- method="als": Specifies the alternating least squares (ALS) optimization method, commonly used for matrix factorization in recommendation systems.

- k=2: Sets the number of latent factors to 2, determining the dimensionality of the latent factor space.

- lambda_=0.1: Sets the regularization strength to 0.1. Regularization is used to prevent overfitting in the model.

- user_bias=False: Indicates that user bias terms are not included in the model. User bias represents a user's overall rating tendency.

- item_bias=False: Excludes item bias terms in the model. Item bias represents an item's overall rating tendency.

- verbose=False: Suppresses verbose output, controlling whether the model's training progress is displayed.

In [ ]:
model.fit(df)

Collective matrix factorization model
(explicit-feedback variant)


In [ ]:
model.A_.shape, model.B_.shape

((668, 2), (10325, 2))

In [ ]:
model.B_

array([[ 0.75743455, -0.06085442],
       [-0.23201275,  1.1658956 ],
       [ 0.6025151 , -0.24058555],
       ...,
       [ 0.10675118,  0.6455457 ],
       [-0.05072206, -0.30672663],
       [ 0.001769  ,  0.01069749]], dtype=float32)

In [ ]:
rm__ = np.dot(model.A_, model.B_.T)
mse(rm.values[rm > 0], rm__[rm > 0]) ** 0.5

3.8516142259951596

In [ ]:
# Homework: Build a classifier/regressor using the users, movies, ratings data

# Created 1 joined dataset
# rating will be the dependent variable
# columns from the tables like age, time_spent, genres etc would be the independent variables

# 80-20 split -> Train-Validation Test